In [ ]:
%load_ext autoreload
%autoreload 2

import sys

from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=['--ngrok_http_tunnel'] if 'google.colab' in sys.modules else [])

In [ ]:
from pydrake.all import (
    ConvexSet, HPolyhedron, HyperEllipsoid,
    MathematicalProgram, Solve, le,
)

# Compute the "center" of a HPolyhedron.
def center(self):
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(self.ambient_dimension())
    r = prog.NewContinuousVariables(1)[0]

    A_row_norm = np.linalg.norm(self.A(), axis=1)
    lhs = self.A().dot(x) + A_row_norm * r
    prog.AddLinearConstraint(le(lhs, self.b()))
    prog.AddLinearConstraint(r >= 0)
    prog.AddLinearCost(-r)

    result = Solve(prog)
    assert result.is_success()

    return result.GetSolution(x)

HPolyhedron.center = center

# TODO: Add a new constructor?  Or a static method?
def HPolyhedron_from_bounds(lb, ub):
    d = len(lb)
    assert len(ub) == d
    I = np.identity(d)
    return HPolyhedron(np.vstack((I, -I)), np.concatenate((ub, -lb)))

def GrowthVolume(self, set):
    # TODO: Robin super-optimized this step for v-rep obstacles (cvxgen, etc).
    assert self.ambient_dimension() == set.ambient_dimension()
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(self.ambient_dimension())
    set.AddPointInSetConstraint(prog, x)
    prog.AddQuadraticErrorCost(self.A().T @ self.A(), self.center(), x)
    result = Solve(prog)
    return result.get_optimal_cost(), result.GetSolution(x)

HyperEllipsoid.GrowthVolume = GrowthVolume

import numpy as np
import meshcat.geometry as g
from pydrake.all import RotationMatrix, RigidTransform
from scipy.spatial import ConvexHull, HalfspaceIntersection

def set_meshcat_object(meshcat, set, color=0xdd9999, wireframe=False):
    material = g.MeshLambertMaterial(color=color, wireframe=wireframe)

    if isinstance(set, np.ndarray):  # Render a point as a sphere.
        meshcat.set_transform(RigidTransform(set).GetAsMatrix4())
        meshcat.set_object(g.Sphere(.01), material)
    elif isinstance(set, HPolyhedron):
        assert set.ambient_dimension() == 3
        halfspaces = np.column_stack((set.A(), -set.b()))
        P = HalfspaceIntersection(halfspaces, set.center())
        vertices = P.intersections
        hull = ConvexHull(vertices)  # orders vertices counterclockwise
        vertices = vertices[hull.vertices]
        faces = hull.simplices
        meshcat.set_object(g.TriangularMeshGeometry(vertices, faces), material)
    elif isinstance(set, HyperEllipsoid):
        shape, X_WG = set.ToShapeWithPose()
        meshcat.set_transform(X_WG.GetAsMatrix4())
        meshcat.set_object(g.Ellipsoid([shape.a(), shape.b(), shape.c()]), material)
    else:
        print(type(set))
        raise NotImplementedError



In [ ]:
from pydrake.all import (
    DiagramBuilder, AddMultibodyPlantSceneGraph, Parser,
    MathematicalProgram, Solve,
    ConnectMeshcatVisualizer,
    Role, Sphere
)
from manipulation.scenarios import AddShape
from manipulation.utils import FindResource

def make_environment(robot=True, goal=True):
    builder = DiagramBuilder()

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    if robot:
        iiwa = AddPlanarIiwa(plant)
        wsg = AddWsg(plant, iiwa, roll=0.0, welded=True)
    if goal:
        sphere = AddShape(plant, Sphere(0.02), "sphere", collision=False)
        X_WO = RigidTransform([0.6, 0, 0.65])
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("sphere"), X_WO)

    parser = Parser(plant)
    shelves = parser.AddModelFromFile(FindResource("models/shelves.sdf"))
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("shelves_body", shelves), RigidTransform([0.6,0,0.4]))

    plant.Finalize()

    visualizer = ConnectMeshcatVisualizer(
        builder,
        scene_graph,
        zmq_url=zmq_url,
        delete_prefix_on_load=False)

    diagram = builder.Build()
    visualizer.load()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

    return diagram, plant, scene_graph

diagram, plant, scene_graph = make_environment(robot=False)
lb = np.array([0, -.5, 0])
ub = np.array([1, .5, 1])

In [ ]:
from meshcat import Visualizer
from pydrake.all import GeometrySet

def iris3d(query, point, lb, ub, require_containment_points=[], termination_threshold=2e-2, iteration_limit=100):
    vis = Visualizer(zmq_url=zmq_url)
    set_meshcat_object(vis['sample'], point, color=0x99dd99)

    ellipsoid_epsilon = 1e-1
    dim = 3
    volume_of_unit_sphere = 4.0*np.pi/3.0
    C = ellipsoid_epsilon*np.eye(3)
    d = point
    best_volume = ellipsoid_epsilon**dim * volume_of_unit_sphere

    inspector = query.inspector()
    num_geometries = inspector.NumGeometriesWithRole(Role.kProximity)

    bounds = HPolyhedron_from_bounds(lb, ub)
    A = np.vstack((np.zeros((num_geometries,3)),bounds.A()))
    b = np.concatenate((np.zeros(num_geometries),bounds.b()))
    A2 = np.copy(A)
    b2 = np.copy(b)

    Cinv = np.linalg.inv(C)
    ellipse = HyperEllipsoid(Cinv, d)
    Cinv2 = Cinv @ Cinv.T
    set_meshcat_object(vis['ellipse'], ellipse)

    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = [HPolyhedron.MakeFromSceneGraph(query, geom) for geom in geom_ids]

    iteration = 0
    while True:

        ## Find separating hyperplanes

        # Closest points
        closest = [ellipse.GrowthVolume(set) for set in sets]
        closest.sort()

        # TODO: Remove redundant planes by checking if the obstacle is already outside the polytope in this loop.
        # Could use AABB to make it fast; the IRIS paper made a mistake, I think, by only checking the points.
        for i,c in enumerate(closest):
            distance, xstar = c
            set_meshcat_object(vis['closest'][f'{i}'], xstar)
            # Tangent planes
            a = 2 * Cinv2 @ (xstar - d)
            A2[i,:] = a / np.linalg.norm(a)
            b2[i] = A2[i,:].dot(xstar)

        #print([A2 @ x > b2 for x in require_containment_points])

        if any([np.any(A2 @ x > b2) for x in require_containment_points]):
            print("terminating because a required containtment point would have not been contained")
            break

        A = A2
        b = b2
        p = HPolyhedron(A,b)
        set_meshcat_object(vis[f'polytope'], p, wireframe=True)

        ## Maximize inscribed ellipse
        prog = MathematicalProgram()
        Cvar = prog.NewSymmetricContinuousVariables(3,'C')
        dvar = prog.NewContinuousVariables(3, 'd')
        prog.AddMaximizeLogDeterminantSymmetricMatrixCost(Cvar)
        prog.AddPositiveSemidefiniteConstraint(Cvar)
        for i in range(len(b)):
            prog.AddLorentzConeConstraint(np.concatenate((
                [b[i] - A[i,:].dot(dvar)],
                Cvar.dot(A[i,:])
            )))
        result = Solve(prog)
        assert result.is_success()
        C = result.GetSolution(Cvar)
        d = result.GetSolution(dvar)

        Cinv = np.linalg.inv(C)
        ellipse = HyperEllipsoid(Cinv, d)
        Cinv2 = Cinv @ Cinv.T

        set_meshcat_object(vis[f'ellipse'], ellipse)
        print(iteration)

        iteration += 1
        if iteration >= iteration_limit:
            break

        volume = np.linalg.det(C) * volume_of_unit_sphere
        if volume - best_volume <= termination_threshold:
            break
        best_volume = volume

    return A,b

np.set_printoptions(formatter={'float': lambda x: "{0:0.1f}".format(x)})
x = np.array([0.5, 0, 0.45])
context = diagram.CreateDefaultContext()
query = scene_graph.get_query_output_port().Eval(scene_graph.GetMyContextFromRoot(context))
iris3d(query, x, lb, ub, require_containment_points=[x], iteration_limit=100);